# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


def nhl_correlation(): 
    # YOUR CODE HERE
    #raise NotImplementedError()
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    nhl_df = nhl_df[nhl_df['year'] == 2018]
    nhl_df = nhl_df[~nhl_df['team'].str.contains('(\.*Division)')]
    nhl_df['team'] = nhl_df['team'].str.replace('[^a-zA-Z\s]','')
    nhl_df = nhl_df.reset_index().drop('index',axis=1)
    city = {'Boston Bruins': 'Boston'
            ,'Ottawa Senators':'Ottawa'
            ,'Dallas Stars':'Dallas'
            ,'Vegas Golden':'Vegas'}
    nhl_df['City'] = nhl_df['team'].str.extract('^(\w.{1,5}(?:\s\w+)?\w*)').replace(city)
    metro_areas = {'Dallas':'Dallas–Fort Worth'
                    ,'Colorado':'Denver'
                    ,'Vegas':'Las Vegas'
                    ,'Anaheim':'Los Angeles'
                    ,'Florida':'Miami–Fort Lauderdale'
                    ,'Minnesota':'Minneapolis–Saint Paul'
                    ,'New York':'New York City'
                    ,'New Jersey':'New York City'
                    ,'Arizona':'Phoenix'
                    ,'Carolina':'Raleigh'
                    ,'San Jose':'San Francisco Bay Area'
                    ,'Tampa Bay':'Tampa Bay Area'
                    ,'Washington':'Washington, D.C.'
                    ,'St Louis':'St. Louis'}
    nhl_df['Metro Area'] = nhl_df['City'].replace(metro_areas)
    nhl_df.sort_values('Metro Area',inplace=True)
    nhl_df = nhl_df.astype({'W':'float','L':'float','GP':'float'})
    nhl_df['Team Win-Loss %'] = nhl_df['W']/nhl_df['GP']

    merged_df = pd.merge(nhl_df, cities, how='inner', left_on='Metro Area', right_on='Metropolitan area')
    merged_df.drop(['Metropolitan area','NFL','MLB','NBA','NHL'], axis=1, inplace=True)

    metro_area_df = merged_df.groupby(['Metro Area','Population (2016 est.)[8]'])[['W','L','GP']].sum()
    metro_area_df['Win-Loss %'] = metro_area_df['W']/(metro_area_df['W']+metro_area_df['L'])
    metro_area_df = metro_area_df.reset_index()
    metro_area_df = metro_area_df.astype({"Population (2016 est.)[8]":"float"})
    
    population_by_region = [i for i in metro_area_df["Population (2016 est.)[8]"]] # pass in metropolitan area population from cities
    win_loss_by_region = [i for i in metro_area_df['Win-Loss %']] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nhl_correlation()

0.012308996455744249

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nba_correlation():
    # YOUR CODE HERE
    # raise NotImplementedError()
    
    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    nba_df = nba_df[nba_df['year'] == 2018]
    nba_df['team'] = nba_df['team'].str.replace('[^a-zA-Z\s]','')
    city = {'Boston Celtics': 'Boston'
            ,'Miami Heat':'Miami'
            ,'Utah Jazz':'Utah'
            ,'Denver Nuggets':'Denver'
            ,'Dallas Mavericks':'Dallas'}
    nba_df['City'] = nba_df['team'].str.extract('^(\w.{1,5}(?:\s\w+)?\w*)').replace(city)
    metro_areas = {'Brooklyn':'New York City'
                   ,'Dallas':'Dallas–Fort Worth'
                   ,'Golden State':'San Francisco Bay Area'
                   ,'Indiana':'Indianapolis'
                   ,'Miami':'Miami–Fort Lauderdale'
                   ,'Minnesota':'Minneapolis–Saint Paul'
                   ,'New York':'New York City'
                   ,'Oklahoma':'Oklahoma City'
                   ,'Utah':'Salt Lake City'
                   ,'Washington':'Washington, D.C.'}
    nba_df['Metro Area'] = nba_df['City'].replace(metro_areas)
    nba_df.sort_values('Metro Area',inplace=True)
    nba_df = nba_df.astype({'W':'float','L':'float'})
    
    merged_df = pd.merge(nba_df, cities, how='inner', left_on='Metro Area', right_on='Metropolitan area')
    merged_df = merged_df.astype({"Population (2016 est.)[8]":"float"})    
    merged_df = merged_df.groupby(['Metro Area','Population (2016 est.)[8]'])[['W','L']].sum()
    merged_df['Win-Loss %'] = merged_df['W']/(merged_df['W']+merged_df['L'])
    merged_df = merged_df.reset_index()
    
    population_by_region = [i for i in merged_df["Population (2016 est.)[8]"]] # pass in metropolitan area population from cities
    win_loss_by_region = [i for i in merged_df['Win-Loss %']] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nba_correlation()

-0.17657160252844617

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [3]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def mlb_correlation(): 
    # YOUR CODE HERE
    # raise NotImplementedError()

    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]  
    mlb_df = mlb_df[mlb_df['year'] == 2018]
    mlb_df['team'] = mlb_df['team'].str.replace('[^a-zA-Z\s\.]','')
    city = {'Boston Red': 'Boston'
            ,'Texas Rangers':'Texas'
            ,'Miami Marlins':'Miami'}  
    mlb_df['City'] = mlb_df['team'].str.extract('^(\w.{1,5}(?:\s\w+)?\w*)').replace(city)
    metro_areas = {'Arizona':'Phoenix'
                   ,'Colorado':'Denver'
                   ,'Miami':'Miami–Fort Lauderdale'
                   ,'Minnesota':'Minneapolis–Saint Paul'
                   ,'New York':'New York City'
                   ,'Oakland':'San Francisco Bay Area'
                   ,'Tampa Bay':'Tampa Bay Area'
                   ,'Texas':'Dallas–Fort Worth'
                   ,'Washington':'Washington, D.C.'}
    mlb_df['Metro Area'] = mlb_df['City'].replace(metro_areas)
    mlb_df.sort_values('Metro Area',inplace=True)
    mlb_df = mlb_df.astype({'W':'float','L':'float'})

    merged_df = pd.merge(mlb_df, cities, how='inner', left_on='Metro Area', right_on='Metropolitan area')
    merged_df = merged_df.astype({"Population (2016 est.)[8]":"float"})    
    merged_df = merged_df.groupby(['Metro Area','Population (2016 est.)[8]'])[['W','L']].sum()
    merged_df['Win-Loss %'] = merged_df['W']/(merged_df['W']+merged_df['L'])
    merged_df = merged_df.reset_index()
    
    population_by_region = [i for i in merged_df["Population (2016 est.)[8]"]] # pass in metropolitan area population from cities
    win_loss_by_region = [i for i in merged_df['Win-Loss %']] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

mlb_correlation()

0.15756084693436004

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def nfl_correlation(): 
    # YOUR CODE HERE
    # raise NotImplementedError()

    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    nfl_df = nfl_df[nfl_df['year'] == 2018]
    nfl_df = nfl_df[~nfl_df['team'].str.contains('(\.*(AFC|NFC)\.*)')]
    nfl_df['team'] = nfl_df['team'].str.replace('[^a-zA-Z\s\.]','')
    city = {'Miami Dolphins': 'Miami'
            ,'Denver Broncos':'Denver'
            ,'Dallas Cowboys':'Dallas'}  
    nfl_df['City'] = nfl_df['team'].str.extract('^(\w.{1,5}(?:\s\w+)?\w*)').replace(city)
    metro_areas = {'New England':'Boston'
                   ,'Miami':'Miami–Fort Lauderdale'
                   ,'New York':'New York City'
                   ,'Tennessee':'Nashville'
                   ,'Oakland':'San Francisco Bay Area'
                   ,'Dallas':'Dallas–Fort Worth'
                   ,'Washington':'Washington, D.C.'
                   ,'Minnesota':'Minneapolis–Saint Paul'
                   ,'Carolina':'Charlotte'
                   ,'Tampa Bay':'Tampa Bay Area'
                   ,'Arizona':'Phoenix'}
    nfl_df['Metro Area'] = nfl_df['City'].replace(metro_areas)
    nfl_df.sort_values('Metro Area',inplace=True)
    nfl_df = nfl_df.astype({'W':'float','L':'float'})

    merged_df = pd.merge(nfl_df, cities, how='inner', left_on='Metro Area', right_on='Metropolitan area')
    merged_df = merged_df.astype({"Population (2016 est.)[8]":"float"})    
    merged_df = merged_df.groupby(['Metro Area','Population (2016 est.)[8]'])[['W','L']].sum()
    merged_df['Win-Loss %'] = merged_df['W']/(merged_df['W']+merged_df['L'])
    merged_df = merged_df.reset_index()
    
    population_by_region = [i for i in merged_df["Population (2016 est.)[8]"]] # pass in metropolitan area population from cities
    win_loss_by_region = [i for i in merged_df['Win-Loss %']] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nfl_correlation()

0.004922112149349393

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [7]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nhl():
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    nhl_df = nhl_df[nhl_df['year'] == 2018]
    nhl_df = nhl_df[~nhl_df['team'].str.contains('(\.*Division)')]
    nhl_df['team'] = nhl_df['team'].str.replace('[^a-zA-Z\s]','')
    city = {'Boston Bruins': 'Boston'
            ,'Ottawa Senators':'Ottawa'
            ,'Dallas Stars':'Dallas'
            ,'Vegas Golden':'Vegas'}
    nhl_df['City'] = nhl_df['team'].str.extract('^(\w.{1,5}(?:\s\w+)?\w*)').replace(city)
    metro_areas = {'Dallas':'Dallas–Fort Worth'
                    ,'Colorado':'Denver'
                    ,'Vegas':'Las Vegas'
                    ,'Anaheim':'Los Angeles'
                    ,'Florida':'Miami–Fort Lauderdale'
                    ,'Minnesota':'Minneapolis–Saint Paul'
                    ,'New York':'New York City'
                    ,'New Jersey':'New York City'
                    ,'Arizona':'Phoenix'
                    ,'Carolina':'Raleigh'
                    ,'San Jose':'San Francisco Bay Area'
                    ,'Tampa Bay':'Tampa Bay Area'
                    ,'Washington':'Washington, D.C.'
                    ,'St Louis':'St. Louis'}
    nhl_df['Metro Area'] = nhl_df['City'].replace(metro_areas)
    nhl_df = nhl_df.astype({'W':'float','L':'float'})
    
    merged_df = pd.merge(nhl_df, cities, how='inner', left_on='Metro Area', right_on='Metropolitan area')
    merged_df = merged_df.astype({"Population (2016 est.)[8]":"float"})    
    #merged_df = merged_df.groupby(['Metro Area','Population (2016 est.)[8]'])[['W','L']].sum()
    merged_df['Win-Loss %'] = merged_df['W']/(merged_df['W']+merged_df['L'])
    merged_df = merged_df.groupby('Metro Area').mean()
    merged_df = merged_df.reset_index()
    
    return merged_df

def nba():
    # YOUR CODE HERE
    # raise NotImplementedError()
    
    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    nba_df = nba_df[nba_df['year'] == 2018]
    nba_df['team'] = nba_df['team'].str.replace('[^a-zA-Z\s]','')
    city = {'Boston Celtics': 'Boston'
            ,'Miami Heat':'Miami'
            ,'Utah Jazz':'Utah'
            ,'Denver Nuggets':'Denver'
            ,'Dallas Mavericks':'Dallas'}
    nba_df['City'] = nba_df['team'].str.extract('^(\w.{1,5}(?:\s\w+)?\w*)').replace(city)
    metro_areas = {'Brooklyn':'New York City'
                   ,'Dallas':'Dallas–Fort Worth'
                   ,'Golden State':'San Francisco Bay Area'
                   ,'Indiana':'Indianapolis'
                   ,'Miami':'Miami–Fort Lauderdale'
                   ,'Minnesota':'Minneapolis–Saint Paul'
                   ,'New York':'New York City'
                   ,'Oklahoma':'Oklahoma City'
                   ,'Utah':'Salt Lake City'
                   ,'Washington':'Washington, D.C.'}
    nba_df['Metro Area'] = nba_df['City'].replace(metro_areas)
    nba_df.sort_values('Metro Area',inplace=True)
    nba_df = nba_df.astype({'W':'float','L':'float'})
    
    merged_df = pd.merge(nba_df, cities, how='inner', left_on='Metro Area', right_on='Metropolitan area')
    merged_df = merged_df.astype({"Population (2016 est.)[8]":"float"})    
    #merged_df = merged_df.groupby(['Metro Area','Population (2016 est.)[8]'])[['W','L']].sum()
    merged_df['Win-Loss %'] = merged_df['W']/(merged_df['W']+merged_df['L'])
    merged_df = merged_df.groupby('Metro Area').mean()
    merged_df = merged_df.reset_index()

    return merged_df

def mlb(): 
    # YOUR CODE HERE
    # raise NotImplementedError()

    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]  
    mlb_df = mlb_df[mlb_df['year'] == 2018]
    mlb_df['team'] = mlb_df['team'].str.replace('[^a-zA-Z\s\.]','')
    city = {'Boston Red': 'Boston'
            ,'Texas Rangers':'Texas'
            ,'Miami Marlins':'Miami'}  
    mlb_df['City'] = mlb_df['team'].str.extract('^(\w.{1,5}(?:\s\w+)?\w*)').replace(city)
    metro_areas = {'Arizona':'Phoenix'
                   ,'Colorado':'Denver'
                   ,'Miami':'Miami–Fort Lauderdale'
                   ,'Minnesota':'Minneapolis–Saint Paul'
                   ,'New York':'New York City'
                   ,'Oakland':'San Francisco Bay Area'
                   ,'Tampa Bay':'Tampa Bay Area'
                   ,'Texas':'Dallas–Fort Worth'
                   ,'Washington':'Washington, D.C.'}
    mlb_df['Metro Area'] = mlb_df['City'].replace(metro_areas)
    mlb_df.sort_values('Metro Area',inplace=True)
    mlb_df = mlb_df.astype({'W':'float','L':'float'})

    merged_df = pd.merge(mlb_df, cities, how='inner', left_on='Metro Area', right_on='Metropolitan area')
    merged_df = merged_df.astype({"Population (2016 est.)[8]":"float"})    
    #merged_df = merged_df.groupby(['Metro Area','Population (2016 est.)[8]'])[['W','L']].sum()
    merged_df['Win-Loss %'] = merged_df['W']/(merged_df['W']+merged_df['L'])
    merged_df = merged_df.groupby('Metro Area').mean()
    merged_df = merged_df.reset_index()

    return merged_df

def nfl(): 
    # YOUR CODE HERE
    # raise NotImplementedError()

    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    nfl_df = nfl_df[nfl_df['year'] == 2018]
    nfl_df = nfl_df[~nfl_df['team'].str.contains('(\.*(AFC|NFC)\.*)')]
    nfl_df['team'] = nfl_df['team'].str.replace('[^a-zA-Z\s\.]','')
    city = {'Miami Dolphins': 'Miami'
            ,'Denver Broncos':'Denver'
            ,'Dallas Cowboys':'Dallas'}  
    nfl_df['City'] = nfl_df['team'].str.extract('^(\w.{1,5}(?:\s\w+)?\w*)').replace(city)
    metro_areas = {'New England':'Boston'
                   ,'Miami':'Miami–Fort Lauderdale'
                   ,'New York':'New York City'
                   ,'Tennessee':'Nashville'
                   ,'Oakland':'San Francisco Bay Area'
                   ,'Dallas':'Dallas–Fort Worth'
                   ,'Washington':'Washington, D.C.'
                   ,'Minnesota':'Minneapolis–Saint Paul'
                   ,'Carolina':'Charlotte'
                   ,'Tampa Bay':'Tampa Bay Area'
                   ,'Arizona':'Phoenix'}
    nfl_df['Metro Area'] = nfl_df['City'].replace(metro_areas)
    nfl_df.sort_values('Metro Area',inplace=True)
    nfl_df = nfl_df.astype({'W':'float','L':'float'})

    merged_df = pd.merge(nfl_df, cities, how='inner', left_on='Metro Area', right_on='Metropolitan area')
    merged_df = merged_df.astype({"Population (2016 est.)[8]":"float"})    
    #merged_df = merged_df.groupby(['Metro Area','Population (2016 est.)[8]'])[['W','L']].sum()
    merged_df['Win-Loss %'] = merged_df['W']/(merged_df['W']+merged_df['L'])
    merged_df = merged_df.groupby('Metro Area').mean()
    merged_df = merged_df.reset_index()

    return merged_df
    
def sports_team_performance():
    # YOUR CODE HERE
    # raise NotImplementedError()
    
    nhl_df = nhl()
    nba_df = nba()
    mlb_df = mlb()
    nfl_df = nfl()
    
    nhl_nba_df = pd.merge(nhl_df,nba_df,how='inner',left_on='Metro Area',right_on='Metro Area')
    nhl_mlb_df = pd.merge(nhl_df,mlb_df,how='inner',left_on='Metro Area',right_on='Metro Area')
    nba_mlb_df = pd.merge(nba_df,mlb_df,how='inner',left_on='Metro Area',right_on='Metro Area')
    nba_nfl_df = pd.merge(nba_df,nfl_df,how='inner',left_on='Metro Area',right_on='Metro Area')
    mlb_nfl_df = pd.merge(mlb_df,nfl_df,how='inner',left_on='Metro Area',right_on='Metro Area')
    nfl_nhl_df = pd.merge(nfl_df,nhl_df,how='inner',left_on='Metro Area',right_on='Metro Area')
    
    pval_nhl_nba = stats.ttest_rel(nhl_nba_df['Win-Loss %_x'],nhl_nba_df['Win-Loss %_y'])
    pval_nhl_mlb = stats.ttest_rel(nhl_mlb_df['Win-Loss %_x'],nhl_mlb_df['Win-Loss %_y'])
    pval_nba_mlb = stats.ttest_rel(nba_mlb_df['Win-Loss %_x'],nba_mlb_df['Win-Loss %_y'])
    pval_nba_nfl = stats.ttest_rel(nba_nfl_df['Win-Loss %_x'],nba_nfl_df['Win-Loss %_y'])
    pval_mlb_nfl = stats.ttest_rel(mlb_nfl_df['Win-Loss %_x'],mlb_nfl_df['Win-Loss %_y'])
    pval_nfl_nhl = stats.ttest_rel(nfl_nhl_df['Win-Loss %_x'],nfl_nhl_df['Win-Loss %_y'])
    
    pval_dict = {'NHL':{'NHL':np.nan,'NBA':pval_nhl_nba[1],'MLB':pval_nhl_mlb[1],'NFL':pval_nfl_nhl[1]}
                 ,'NBA':{'NHL':pval_nhl_nba[1],'NBA':np.nan,'MLB':pval_nba_mlb[1],'NFL':pval_nba_nfl[1]}
                 ,'MLB':{'NHL':pval_nhl_mlb[1],'NBA':pval_nba_mlb[1],'MLB':np.nan,'NFL':pval_mlb_nfl[1]}
                 ,'NFL':{'NHL':pval_nfl_nhl[1],'NBA':pval_nba_nfl[1],'MLB':pval_mlb_nfl[1],'NFL':np.nan}}
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    #sports = ['NFL', 'NBA', 'NHL', 'MLB']
    #p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    p_values = pd.DataFrame(pval_dict)
    
    return p_values

sports_team_performance()

,NHL,NBA,MLB,NFL
NHL,NaN,0.022297,0.001234,0.030883
NBA,0.022297,NaN,0.959384,0.941792
MLB,0.001234,0.959384,NaN,0.866559
NFL,0.030883,0.941792,0.866559,NaN
